# Подготавливаем данные для второго этапа предсказания

In [11]:
from utils import *

### Посмотрим на предсказания, полученные обучением на 5 отдельных моделях:

In [12]:
data = np.load('./data/predictions.npy')
print(data) 

[list(['S-ORG', 'O', 'S-MISC', 'O', 'O', 'O', 'S-MISC', 'O', 'O'])
 list(['B-PER', 'E-PER']) list(['S-LOC', 'O']) ...
 list(['S-ORG', 'O', 'S-ORG', 'O']) list(['O', 'O'])
 list(['S-ORG', 'O', 'S-ORG', 'O'])]


## Предсказания у нас сейчас в виде предложений, а нужно побить на документы!

In [13]:
import utils
x_train_docs, y_train_docs = docs_from_dataset('./data/conll', 'eng.train.txt', 
                                                ('words', 'pos', 'chunk', 'ne'), 
                                                ['words', 'pos', 'chunk'], sent2features)

### Разбиваем предсказания на документы:

In [14]:
index = 0
predictions_as_docs = []
for doc in y_train_docs:
    new_pred_doc = []
    for sent in doc:
        new_pred_doc.append(data[index])
        index += 1
    predictions_as_docs.append(new_pred_doc)

### Вычисляем новые признаки и добавляем их к документам:

In [15]:
add_token_features(x_train_docs, predictions_as_docs)
add_entity_features(x_train_docs, predictions_as_docs)
add_super_features(x_train_docs, predictions_as_docs)

### Полученные признаки сохраняем в формате, который используется в нейросети:

#### Говорим энкодеру считать пустой тег за 0, тогда pad_sequence будет работать адекватно.

In [16]:
from sklearn.preprocessing import OneHotEncoder
from model.config import Config

config = Config()
encoder = utils.LabelEncoder()

encoder.get('O')

def get_features(x_data):
    return [
        x_data.get('doc_token_maj', 'O'),
        x_data.get('corp_token_maj', 'O'),
        x_data.get('doc_entity_maj', 'O'),
        x_data.get('doc_entity_maj', 'O'),
        x_data.get('corp_super_maj', 'O'),
        x_data.get('corp_super_maj', 'O')
    ]
    
dataset_formatted = []

for x_doc, y_doc in zip(x_train_docs, y_train_docs):
    for x_sent, y_sent in zip(x_doc, y_doc):
        words = []
        tags = []
        features = []
        for x_data, y_token in zip(x_sent, y_sent):
            additional_features = get_features(x_data)
            words.append(config.processing_word(x_data['word']))
            tags.append(config.processing_tag(y_token))
            features.append([encoder.get(tag) for tag in additional_features])
        dataset_formatted.append([words, features, tags])
        
only_features = [sent[1] for sent in dataset_formatted]
only_features_flat = []

for sent_features in only_features:
    only_features_flat += sent_features

enc = OneHotEncoder()
enc.fit(only_features_flat)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

### Посмотрим, как выглядят наши признаки в первом предложении:

In [17]:
print(dataset_formatted[0])

[[([25, 22], 2881), ([0, 76, 39, 76, 32, 64, 31], 161), ([4, 76, 0, 40, 14, 33], 13239), ([32, 14, 59, 59], 13603), ([64, 71], 5382), ([67, 71, 47, 32, 71, 64, 64], 12252), ([26, 0, 21, 64, 21, 31, 3], 781), ([59, 14, 40, 67], 17131), ([60], 16992)], [[1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [2, 2, 2, 2, 3, 3], [0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 4, 4], [0, 0, 0, 0, 0, 0], [2, 2, 2, 2, 2, 2], [0, 0, 0, 0, 4, 4], [0, 0, 0, 0, 3, 3]], [9, 13, 11, 13, 13, 13, 11, 13, 13]]


### Применим к ним обученный OneHotEncoder:

In [18]:
for sent in dataset_formatted:
    sent[1] = [enc.transform([word_features]).toarray()[0].tolist() for word_features in sent[1]]

### Проверим, как они выглядят теперь:

In [19]:
print(dataset_formatted[0])

[[([25, 22], 2881), ([0, 76, 39, 76, 32, 64, 31], 161), ([4, 76, 0, 40, 14, 33], 13239), ([32, 14, 59, 59], 13603), ([64, 71], 5382), ([67, 71, 47, 32, 71, 64, 64], 12252), ([26, 0, 21, 64, 21, 31, 3], 781), ([59, 14, 40, 67], 17131), ([60], 16992)], [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,

### Сохраним полученный результат:

In [21]:
np.save('./data/dataset_formatted', np.array(dataset_formatted))